In [ ]:
import sys
sys.path.append('../')

from cube import Cube
import numpy as np
from tqdm import tqdm
import torch
from copy import deepcopy
import matplotlib.pyplot as plt
from value_policy_net import ValuePolicyNet
from resnet_model import ResnetModel


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
k = 15
l = 100
M = 20

lr = 1e-4
batch_len = 32
epochs_per_iteration = 1
reward_type = 'deepcube' # what reward type to use, makes a difference only if value_only = False (DeepCube / MCTS). Values are 'deepcube' or 'dqn'.
network_type = 'resnet' # what network type to use, makes a difference only if value_only = True (DeepCubeA / A*). Values are 'normal' or 'resnet'.
value_only = True # value_only False trains the model for DeepCube (mcts), while True trains the model for DeepCubeA (A*)

In [ ]:
action_encode = {
    'F': 0,
    'F\'': 1,
    'B': 2,
    'B\'': 3,
    'L': 4,
    'L\'': 5,
    'R': 6,
    'R\'': 7,
    'U': 8,
    'U\'': 9,
    'D': 10,
    'D\'': 11,
}
action_decode = {encoding: action for action,
                 encoding in action_encode.items()}

In [ ]:
vp_net = ValuePolicyNet(value_only = value_only).to(device) if network_type == 'normal' else ResnetModel(20, 24, 5000, 1000, 4, 1, True).to(device)
optim = torch.optim.RMSprop(vp_net.parameters(), lr=lr)

In [ ]:
def get_scrambled_cubes(k, l):
    states = []
    for _ in range(l):
        cube = Cube()
        scramble_str = cube.get_scramble(k)
        for rot_num, rot_code in enumerate(scramble_str.split(' ')):
            cube.rotate_code(rot_code)
            states.append(
                (np.copy(cube.facelets), np.copy(cube.tracked), rot_num + 1))

    return states


def batchify(X, Y, W, batch_size):
    x_batch = []
    y_v_batch = []
    y_p_batch = []
    w_batch = []
    for x, (y_v, y_p), w in zip(X, Y, W):
        x_batch.append(x)
        y_v_batch.append(y_v)
        y_p_batch.append(y_p)
        w_batch.append(w)
        if len(x_batch) >= batch_size:
            if value_only:
                yield torch.Tensor(np.array(x_batch)).to(device), torch.Tensor(y_v_batch).to(device), None, torch.Tensor(w_batch).to(device)
            else:
                yield torch.Tensor(np.array(x_batch)).to(device), torch.Tensor(y_v_batch).to(device), torch.Tensor(y_p_batch).to(device), torch.Tensor(w_batch).to(device)
            x_batch.clear()
            y_v_batch.clear()
            y_p_batch.clear()
            w_batch.clear()
    if len(x_batch) > 0:
        if value_only:
            yield torch.Tensor(np.array(x_batch)).to(device), torch.Tensor(y_v_batch).to(device), None, torch.Tensor(w_batch).to(device)
        else:
            yield torch.Tensor(np.array(x_batch)).to(device), torch.Tensor(y_v_batch).to(device), torch.Tensor(y_p_batch).to(device), torch.Tensor(w_batch).to(device)
    return None

In [ ]:
losses = []
for it in range(M):
    cube = Cube()
    scrambled_cubes = get_scrambled_cubes(k, l)
    F = [f for f, _, _ in scrambled_cubes]
    T = [t for _, t, _ in scrambled_cubes]
    W = [1 / n for _, _, n in scrambled_cubes]
    X = [Cube.encode_state(t, cube.edges_corners) for t in T]
    Y = []

    vp_net.eval()
    for f, t in zip(F, T):
        v_x = []
        r_x = []
        solved = []
        for a in range(12):
            cube.facelets = np.copy(f)
            cube.tracked = np.copy(t)
            r_x_a = cube.rotate_code_get_reward(action_decode[a], reward_type)
            with torch.no_grad():
                net_out = vp_net(torch.Tensor(Cube.encode_state(cube.tracked, cube.edges_corners))[None, :].to(device))
                # Even in the case of both value and policy, we only care about the value of the next state
                v_x_a = net_out[0] if value_only else net_out[0][0]
            solved.append(cube.is_solved())

            v_x.append(v_x_a)
            r_x.append(r_x_a)

        v_x = torch.Tensor(v_x)
        if not value_only: # For value and policy we set v_next to the reward if solved
            r_x = torch.Tensor(r_x)

            v_next = r_x + v_x
            for a in range(12):
                if solved[a]:
                    v_next[a] = r_x[a]
        else: # For value only, we don't care about rewards, as value is actually the approximated number of steps to solve
            v_next = 1 + v_x
            
            for a in range(12):
                if solved[a]:
                    v_next[a] = 1 # TODO: makes more sense to be 1 here but what if it's actually 0? Maybe skim the paper again to check
                
        if not value_only: # For value and policy we get the max of the next values
            y_v = torch.max(v_next)
            y_p = torch.argmax(v_next)
            Y.append((y_v, y_p))
        else: # For value only we get the min of the next values
            y_v = torch.min(v_next)
            Y.append((y_v, None))
    vp_net.train()

    shuffle_indices = np.arange(len(X))
    # np.random.shuffle(shuffle_indices)
    X = [X[i] for i in shuffle_indices]
    Y = [Y[i] for i in shuffle_indices]
    W = [W[i] for i in shuffle_indices]
    
    it_losses = []
    for epoch in range(epochs_per_iteration):
        epoch_losses = []
        for x_batch, y_v_batch, y_p_batch, w_batch in batchify(X, Y, W, batch_len):
            optim.zero_grad()
            if not value_only: # For value and policy we use both the loss of value and policy
                y_v_pred, y_p_pred = vp_net(x_batch)
                v_loss = torch.mean(w_batch * (y_v_batch - y_v_pred) ** 2)
                p_loss = torch.mean(
                    w_batch * torch.nn.CrossEntropyLoss(reduction='none')(y_p_pred, y_p_batch.long()))
                loss = v_loss + p_loss
            else: # For value only we use only the loss of value
                y_v_pred = vp_net(x_batch)
                v_loss = torch.mean(w_batch * (y_v_batch - y_v_pred) ** 2)
                loss = v_loss
            loss.backward()
            optim.step()

            epoch_losses.append(loss.item())
        it_losses.append(np.mean(epoch_losses))
    it_loss = np.mean(it_losses)
    losses.append(it_loss)

    print(f'Iteration {it}: loss = {it_loss}')

In [ ]:
plt.plot(losses)
plt.title('Average loss per iteration')
ax = plt.gca()
ax.set_ylim([0, 2])
plt.show()

In [ ]:
torch.save(vp_net.state_dict(), 'vp_net.pt' if not value_only else f'v_net_{network_type}.pt')